### combine pfak data seg into npy and tiff, getting weight ready

Liya Ding, 2024.06

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_closing, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [3]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

In [4]:
image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_zyxin/Y27_images'
zyxin_seg_output_dir = '/mnt/d/lding/FA/analysis_results/zyxin_zyxin_20230423/Y27_zyxin/Vessp5-LocThrk1_seg/seg'
pfak_seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Y27_pfak/Vessp5-LocThrk1_seg/seg'
mask_seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Y27_zyxin/Vessp5-LocThrk1_seg/mask'
combined_npy_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/combined_npy_Y27'
combined_tiff_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/combined_tiff_Y27'

if not os.path.isdir(combined_npy_output_dir):   os.makedirs(combined_npy_output_dir)
if not os.path.isdir(combined_tiff_output_dir):  os.makedirs(combined_tiff_output_dir)


In [5]:
image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/Control_images'
zyxin_seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_zyxin/Vessp5-LocThrk1_seg/seg'
pfak_seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_pFak/Vessp5-LocThrk1_seg/seg'
mask_seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_zyxin/Vessp5-LocThrk1_seg/mask'
combined_npy_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/combined_npy_control'
combined_tiff_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/combined_tiff_control'

if not os.path.isdir(combined_npy_output_dir):   os.makedirs(combined_npy_output_dir)
if not os.path.isdir(combined_tiff_output_dir):  os.makedirs(combined_tiff_output_dir)


In [6]:
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
   
    filename = filenames[filenameID]
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    zyxin_img = img[0,:,:]
    pfak_img = img[1,:,:]

    if(filenameID==0):
        pfak_img_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])
        zyxin_img_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])
        pfak_img_in_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])
        zyxin_img_in_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])
        pfak_img_out_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])
        zyxin_img_out_all = np.zeros([len(filenames),img.shape[1],img.shape[2]])

    
    new_cell_mask = tifffile.imread(os.path.join(mask_seg_output_dir,'SS_cell_mask_'+filename+'_MIP'+'.tif'))
    

    pfak_img_all[filenameID,:,:] = pfak_img
    zyxin_img_all[filenameID,:,:] = zyxin_img


    # zyxin_seg = tifffile.imread(os.path.join(zyxin_seg_output_dir,'SS_pax_seg_'+filename+'_MIP'+'.tif'))
    # pfak_seg = tifffile.imread(os.path.join(pfak_seg_output_dir,'SS_pax_seg_'+filename+'_MIP'+'.tif'))

    pfak_img_in_all[filenameID,:,:] = pfak_img*new_cell_mask
    zyxin_img_in_all[filenameID,:,:] = zyxin_img*new_cell_mask
    pfak_img_out_all[filenameID,:,:] = pfak_img*(1-new_cell_mask)
    zyxin_img_out_all[filenameID,:,:] = zyxin_img*(1-new_cell_mask)




In [42]:
from scipy import stats
pfak_out = pfak_img_out_all[pfak_img_out_all>0]
pfak_out = pfak_out[pfak_out<np.percentile(pfak_out,99)]
pfak_back_mode = stats.mode(pfak_out)
pfak_min = pfak_back_mode.mode[0] - 3*pfak_out.std()

pfak_inside = pfak_img_in_all[pfak_img_in_all>0]
pfak_inside = pfak_inside[pfak_inside<np.percentile(pfak_inside,99)]
pfak_insize_mode = stats.mode(pfak_inside)

pfak_max = pfak_insize_mode.mode[0] + 5*(np.percentile(pfak_inside,75) - np.percentile(pfak_inside,25)) + 5*(np.percentile(pfak_inside,75) - np.percentile(pfak_inside,50))


print([pfak_min,pfak_max])


[95.79950438409017, 314.0]


In [43]:
from scipy import stats
zyxin_out = zyxin_img_out_all[zyxin_img_out_all>0]
zyxin_out = zyxin_out[zyxin_out<np.percentile(zyxin_out,99)]
zyxin_back_mode = stats.mode(zyxin_out)
zyxin_min = zyxin_back_mode.mode[0] - 3*zyxin_out.std()

zyxin_inside = zyxin_img_in_all[zyxin_img_in_all>0]
zyxin_inside = zyxin_inside[zyxin_inside<np.percentile(zyxin_inside,99)]
zyxin_insize_mode = stats.mode(zyxin_inside)

zyxin_max = zyxin_insize_mode.mode[0] + 5*(np.percentile(zyxin_inside,75) - np.percentile(zyxin_inside,25)) + 5*(np.percentile(zyxin_inside,75) - np.percentile(zyxin_inside,50))


print([zyxin_min,zyxin_max])

[92.16788260249456, 513.0]


In [44]:
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
   
    filename = filenames[filenameID]
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    zyxin_img = img[0,:,:]
    pfak_img = img[1,:,:]

    zyxin_img = (zyxin_img - zyxin_min)/(zyxin_max-zyxin_min)
    zyxin_img[zyxin_img<0]=0
    zyxin_img[zyxin_img>1]=1

    pfak_img = (pfak_img - pfak_min)/(pfak_max-pfak_min)
    pfak_img[pfak_img<0]=0
    pfak_img[pfak_img>1]=1   

    
    new_cell_mask = tifffile.imread(os.path.join(mask_seg_output_dir,'SS_cell_mask_'+filename+'_MIP'+'.tif'))
    zyxin_seg = tifffile.imread(os.path.join(zyxin_seg_output_dir,'SS_pax_seg_'+filename+'_MIP'+'.tif'))
    pfak_seg = tifffile.imread(os.path.join(pfak_seg_output_dir,'SS_pax_seg_'+filename+'_MIP'+'.tif'))

    pfak_seg_masked = pfak_seg* new_cell_mask
    zyxin_seg_masked = zyxin_seg* new_cell_mask

    union_img = (pfak_seg_masked + zyxin_seg_masked) > 0 
    diff_img = np.abs(pfak_seg_masked - zyxin_seg_masked) > 0

    mask_weight =  np.zeros([img.shape[1],img.shape[2]])
    
    mask_weight[new_cell_mask>0] = 1
    mask_weight[union_img>0] = 2
    mask_weight[diff_img>0] = 3    

    new_img = np.zeros([8,img.shape[1],img.shape[2]])

    new_img[6,:,:] = zyxin_img
    new_img[7,:,:] = pfak_img
    new_img[4,:,:] = new_cell_mask

    new_img[3,:,:] = mask_weight
    new_img[2,:,:] = pfak_seg_masked
    new_img[1,:,:] = zyxin_seg_masked
    new_img[0,:,:] = diff_img
    new_img[5,:,:] = union_img
    

    np.save(os.path.join(combined_npy_output_dir, 'combined_'+filename[:-4]+'.npy'),new_img)   
    tifffile.imsave(os.path.join(combined_tiff_output_dir, 'combined_'+filename[:-4] +'.tif'),new_img.astype(np.float32), metadata={'axes':'CXY'})   
      
    
                